# PointConv: Deep Convolutional Networks on 3D Point Clouds

## Paper Reviews

### Prior Reseach

**PointNet** Point Cloud data를 다루기 위해 permutation invariant한 max-pooling으로 이를 해결했음  
하지만 이는 global region에서의 max response 만을 check하게 되어 local한 semantic을 놓치게 됨   
이를 위해 **PointNet++** 는 hierarchical한 구조를 만들어 냈으나 여전히 local region 내부에서는 PointNet Architecture가 적용되어 있음      
→ local region의 semantic을 손실없이 check하는 Architecture를 고려할 필요가 있음

<p align="center">
<img width="423" alt="1" src="https://user-images.githubusercontent.com/86907286/194721224-f809c67f-418b-4375-9893-b81403c55811.png">
</p>


일반적인 2D image에서 이러한 local representation을 얻는 방법은 **convolution**이라고 할 수 있음  
그러나 Point Cloud는 non-uniform하게 분포하고 있으므로 2D image에서처럼 **fixed lattice**에 적용되는 Convolution layer를 활용할 수 없음  
→ 이를 가장 쉽게 해결할 수 있는 방법은 **Continuous**한 convolution kernel을 estimate하여 이를 적용하는 것  

∴ MLP를 통해 Continuous convolution kernel을 학습하고 이에 대한 operation을 **Monte Carlo sampling**처럼 생각하는 **PointConv**를 제안

<p align="center">
  <img width="423" alt="2" src="https://user-images.githubusercontent.com/86907286/194721226-b0e68652-e8ea-4981-99a8-a477dfd69e41.png">
</p>

### Convolution on 3D Point Clouds

point $p = (x, y, z)$에 대한 local region $G$에서의 feature $F$의 continuous kernel $W$의 convolution은 다음과 같이 나타낼 수 있음  
이때 $(\delta_x, \delta_y, \delta_z)$는 local region $G$에 속한 local point가 target point $p$를 중심으로 떨어진 정도를 나타냄  

$$ \iiint_{(\delta_x, \delta_y, \delta_z) \in G} W(\delta_x, \delta_y, \delta_z) F(x+\delta_x, y+\delta_y, z+\delta_z) d\delta_x \delta_y \delta_z $$




하지만 실제로 우리가 local region $G$에서 얻을 수 있는 것은 **sample point** 이며 심지어 **non-uniform** 함  
→ Monte Carlo integral로 estimate한다면 sample point $x_i$에 대해서 **inverse density scale** $S(\delta_x, \delta_y, \delta_z) = \frac{1}{p(x_i)}$를 또한 근사할 필요가 있음

$$ \int f(x)dx = \int \frac{f(x)}{p(x)}p(x)dx \approx \sum_i \frac{f(x)}{p(x)}p(x) \text{  where  } x_i \sim p(x) $$

따라서 각 local point의 density를 알아야 하는데 이를 **Kernel Density Estimation**를 활용할 수 있음      
이를 통해서 각 kernel의 density를 나타내는 1D vector를 얻은 후 nonlinear transform을 적용하는 MLP를 적용하여 density scale $S(k)$를 얻음  

이렇게 얻은 $S(k)$과 point마다 공유되는 kernel weight $\mathbf{W}(k, c_{\text{in}})$ 두개로 **PointConv** layer를 구성  
이를 PointNet++의 Set Abstraction Block의 PointNet layer를 대체하는 것으로 더 좋은 local representation aggregator가 달성될 수 있음  

$$ \mathbf{F}_{\text{out}} = \sum^K_{k=1} \sum^{C_{\text{in}}}_{c_{\text{in}} = 1} S(k) \mathbf{W}(k, c_{\text{in}}) F_{\text{in}}(k, c_{\text{in}}) $$

<p align="center">
  <img width="874" alt="3" src="https://user-images.githubusercontent.com/86907286/194721229-b8151b79-ba3f-467a-8d23-537dba4ee558.png">
</p>

### Feature Propagation Using Deconvolution

PointNet++이 그랬듯 Segmentation을 수행하기 위해서는 subsampled point에서의 feature propagation이 필요함  
그러나 PointNet++ 처럼 distance-based linear interpolation은 local context의 correlation을 propagation 하는데 한계가 있음  

따라서 PointConv가 일종의 convolution operation에 대한 estimation이듯이 **deconvolution**으로 upsampling하는 것이 적절할 수 있음  
2D Segmentation에서 활용된 **Deconvolution layer**처럼 coarse/fine feature를 concatenate하여 upsampling을 수행  
→ 3개의 neighborhood point에 대해 interpolate 한 후 같은 resolution의 feature를 concatenate하고 PointConv 

<p align="center">
  <img width="441" alt="4" src="https://user-images.githubusercontent.com/86907286/194721234-144076a6-7614-4938-9d3c-cf0f7f94a596.png">
</p>

### Efficient PointConv

단순히 Monte Carlo integral form을 만족시키기 위해 구성한 Architecture는 굉장히 많은 memory를 소모하게 됨  
이는 실질적으로 feature $F$에 곱해지기 위한 kernel weight $\mathbf{W}$를 **전부 계산하기 때문**임  
→ kernel function을 근사하는 MLP는 point마다 공유되어도 function value $\mathbf{W}(k, c_{\text{in}})$은 다를 수 밖에 없음


그러나 PointConv의 **마지막에 전체 point에 대한 summation이 발생하므로 $K$에 대한 summation을 먼저 수행**하는 방법을 쓸 수 있음   
이때 $\mathbf{W}$는 중간 layer output $\mathbf{M} \in \mathbb{R}^{K \times C_{\text{mid}}}$에 대해 마지막 layer weight인 $\mathbf{H} \in \mathbb{R}^{C_{\text{mid}} \times (C_{\text{in}} \times C_{\text{out}})}$로 1x1 Convolution을 수행한 $\text{Conv}_{1\times1}(\mathbf{H}, \mathbf{M})$ 라고 할 수 있음  
이는 결과적으로 $\mathbf{W}$의 계산 과정에서 $K \times C_{\text{out}}$ dimension이 $\mathbf{W}$의 중간 layer output $\mathbf{M}$의 dimension $C_{\text{mid}}$ 로 대체되어 효율적
 
$$ \mathbf{F}_{\text{out}} = \sum^K_{k=1} \text{Conv}_{1\times1}(\mathbf{H}, \mathbf{M}) \odot (S \cdot \mathbf{F}_{\text{in}})_{\times C_{\text{out}}} = \text{Conv}_{1\times1}(\mathbf{H}, \sum^K_{k=1}(\mathbf{M} \odot (S \cdot \mathbf{F}_{\text{in}})_{\times C_{\text{out}}})) = \text{Conv}_{1\times1}(\mathbf{H}, (S \cdot \mathbf{F}_{\text{in}}) \cdot \mathbf{M}) $$

<p align="center">
  <img width="701" alt="5" src="https://user-images.githubusercontent.com/86907286/194721235-13fe8ebd-ef5b-4b5e-8154-5e318ebd2afc.png">
</p>

## Implementation Reviews

Set Abstraction with PointConv

In [ ]:
class PointConvDensitySetAbstraction(nn.Module):
    def __init__(self, npoint, nsample, in_channel, mlp, bandwidth, group_all):
        super(PointConvDensitySetAbstraction, self).__init__()
        self.npoint = npoint
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel

        self.weightnet = WeightNet(3, 16)
        self.linear = nn.Linear(16 * mlp[-1], mlp[-1])
        self.bn_linear = nn.BatchNorm1d(mlp[-1])
        self.densitynet = DensityNet()
        self.group_all = group_all
        self.bandwidth = bandwidth

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        B = xyz.shape[0]
        N = xyz.shape[2]
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        xyz_density = compute_density(xyz, self.bandwidth)
        inverse_density = 1.0 / xyz_density 

        if self.group_all:
            new_xyz, new_points, grouped_xyz_norm, grouped_density = sample_and_group_all(xyz, points, inverse_density.view(B, N, 1))
        else:
            new_xyz, new_points, grouped_xyz_norm, _, grouped_density = sample_and_group(self.npoint, self.nsample, xyz, points, inverse_density.view(B, N, 1))
        # new_xyz: sampled points position data, [B, npoint, C]
        # new_points: sampled points data, [B, npoint, nsample, C+D]
        new_points = new_points.permute(0, 3, 2, 1) # [B, C+D, nsample,npoint]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points =  F.relu(bn(conv(new_points)))

        inverse_max_density = grouped_density.max(dim = 2, keepdim=True)[0]
        density_scale = grouped_density / inverse_max_density
        density_scale = self.densitynet(density_scale.permute(0, 3, 2, 1))
        new_points = new_points * density_scale

        grouped_xyz = grouped_xyz_norm.permute(0, 3, 2, 1)
        weights = self.weightnet(grouped_xyz)     
        new_points = torch.matmul(input=new_points.permute(0, 3, 1, 2), other = weights.permute(0, 3, 2, 1)).view(B, self.npoint, -1)
        new_points = self.linear(new_points)
        new_points = self.bn_linear(new_points.permute(0, 2, 1))
        new_points = F.relu(new_points)
        new_xyz = new_xyz.permute(0, 2, 1)

        return new_xyz, new_points

## Reference

https://arxiv.org/abs/1811.07246  
https://github.com/vinits5/learning3d/blob/28c834e51ef54af23ca49ed3be069c8ff0d906e2/utils/pointconv_util.py